In [ ]:
# Esta celda actúa sobre la libreta, y permite que se aproveche mejor el espacio.
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:95%}</style>"))
import sys
print(sys.executable) 
#import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import seaborn as sns
from sklearn import preprocessing

from IPython.display import HTML
HTML('''<script>
code_show=false; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
El codigo Python de este cuaderno está oculto para facilitar la lectura.
Para mostrar/ocultar el código, haz click <a href="javascript:code_toggle()">aquí.</a>.''')

# Load results

In [ ]:
import pandas as pd
df_alg=pd.read_csv('../output/metrics2307/merged_output_genetic.txt')
convert_dict = {'Dataset': "string", 
                'Algorithm': "string",
                'Population Length': "int64",
                'Generations': "int64",
                'Selection Scheme': "string",
                'Selection Candidates': "int64",
                'Crossover Scheme': "string",
                'Crossover Probability': "float64",
                'Mutation Scheme': "string",
                'Mutation Probability': "float64",
                'Replacement Scheme': "string",
                'Time(s)': "float64",
                'AvgValue': "float64",
                'BestAvgValue': "float64",
                'BestGeneration': "int64",
                'HV': "float64",
                'Spread': "float64",
                'NumSolutions': "float64",
                'Spacing': "float64",
                'NumGenerations': "int64"
               } 
df_alg = df_alg.astype(convert_dict) 
df_alg.loc[(df_alg.Dataset == 'dataset1'),'Dataset']='1'
df_alg.loc[(df_alg.Dataset == 'dataset2'),'Dataset']='2'
display(df_alg.head(2))

df_alg2=pd.read_csv('../output/metrics2307/merged_output_grasp.txt',header=0)
convert_dict = {'Dataset': "string", 
                'Algorithm': "string",
                'Iterations': "float64",
                'Solutions per Iteration': "int64",
                'Initialization Type': "string",
                'Local Search Type': "string",
                'Path Relinking': "string",
                'Time(s)': "float64",
                'AvgValue': "float64",
                'BestAvgValue': "float64",
                'HV': "float64",
                'Spread': "float64",
                'NumSolutions': "int64",
                'Spacing': "float64",
                'NumGenerations': "int64"
               } 
df_alg2 = df_alg2.astype(convert_dict)
display(df_alg2.head(2))

df_alg = df_alg.append(df_alg2)
display(df_alg.head(2))

In [ ]:
def findData(algorithm,type):
    if type=="genetic":
        df_a = df_alg[(df_alg["Algorithm"]=="NSGAIIAlgorithm")&(df_alg["Dataset"]==algorithm.dataset_name)&(df_alg["Population Length"]==algorithm.population_length)
                     &(df_alg["Generations"]==algorithm.max_generations)&(df_alg["Selection Scheme"]==algorithm.selection_scheme)
                     &(df_alg["Selection Candidates"]==algorithm.selection_candidates)&(df_alg["Crossover Scheme"]==algorithm.crossover_scheme)
                     &(df_alg["Crossover Probability"]==algorithm.crossover_prob)&(df_alg["Mutation Scheme"]==algorithm.mutation_scheme)
                     &(df_alg["Mutation Probability"]==algorithm.mutation_prob)&(df_alg["Replacement Scheme"]==algorithm.replacement_scheme)]
    elif type=="grasp":
        df_a = df_alg[(df_alg["Algorithm"]=="GRASP")&(df_alg["Dataset"]==algorithm.dataset_name)&(df_alg["Iterations"]==algorithm.iterations)
                     &(df_alg["Solutions per Iteration"]==algorithm.solutions_per_iteration)&(df_alg["Initialization Type"]==algorithm.init_type)
                     &(df_alg["Local Search Type"]==algorithm.local_search_type)&(df_alg["Path Relinking"]==algorithm.path_relinking_mode)]
    #display(df_a['Time(s)'])
    df_a=(
        df_a[['Time(s)', 'HV', 'Spread',"AvgValue","BestAvgValue",
              #"BestGeneration",
              "NumSolutions","Spacing"]]\
        .mean()
        #.reset_index()\
        #.sort_values("mean_avgvalue",ascending=False)
    )
    return df_a

In [ ]:
#import matplotlib.pyplot as plt
#import numpy as np
from algorithms.GRASP.GRASP import GRASP
from algorithms.genetic.nsgaii.nsgaii_algorithm import NSGAIIAlgorithm

sizes=[50,45,40,35,30,25]
dataset="s3"
seed=10
generations=300
solutions_per_iteration=500
population_length=200
algorithms = [
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="uniform",local_search_type="None",
          path_relinking_mode="None"),
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="stochastically",local_search_type="None",
          path_relinking_mode="None"),
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="stochastically",local_search_type="None",
          path_relinking_mode="after_local"),
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="stochastically",local_search_type="best_first_neighbor_sorted_score",
          path_relinking_mode="None"),
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="stochastically",local_search_type="best_first_neighbor_sorted_score",
          path_relinking_mode="after_local"),
    NSGAIIAlgorithm(dataset_name=dataset,random_seed=seed,population_length=population_length,max_generations=generations,crossover_prob=0.8,
						   crossover="onepoint",mutation_prob=1.0,mutation="flip1bit",replacement="elitism"),
]

#fig, (ax1, ax2) = plt.subplots(1, 2)
#fig.suptitle("Dataset "+dataset)
#ax1.plot(x, y)
#ax2.plot(x, -y)

labels=[]
data_bar=[]
for i in range(len(algorithms)):   
    #print(algorithms[i].get_name())
    if "GRASP" in algorithms[i].file:
        pareto_file = "../output/output/pareto-grasp-"+algorithms[i].file
        metrics_file = "../output/metrics2307/pareto-grasp-"+algorithms[i].file
        df = findData(algorithms[i],"grasp")
        #display(df)
    else:
        pareto_file = "../output/output/pareto-genetic-"+algorithms[i].file
        metrics_file = "../output/metrics2307/pareto-genetic-"+algorithms[i].file
        df = findData(algorithms[i],"genetic")
        #display(df)
    data = np.loadtxt(pareto_file,delimiter=',', dtype=float)
    x,y=data.T
    #ax1.scatter(x,y,label=algorithms[i].get_name(),s=sizes[i])
    plt.scatter(x,y,label=algorithms[i].get_name(),s=sizes[i])
    labels.append(algorithms[i].get_name())
    #display(df["Time(s)"])
    data_bar.append(df["Time(s)"])
#ax2.bar(["labels","a"],[4,2],width=0.2)
#print(labels)
#print(data_bar)
#bar_ax=ax2.bar(labels,data_bar,width=0.2)
#ax2.bar_label(bar_ax, label_type='center')
#for i, v in enumerate(data_bar):
#    plt.text(labels[i], v, str(v))

#plt.set(xlabel='Coste', ylabel='Satisfacción')
#plt.set(xlabel='Algoritmo', ylabel='Tiempo(s)')
plt.xlabel('Coste', fontsize=12)
plt.ylabel('Satisfacción', fontsize=12)
plt.legend(loc="lower right")
plt.grid(True)
plt.rcParams['figure.figsize'] = [26, 10]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.show()

In [ ]:
#import matplotlib.pyplot as plt
#import numpy as np
from algorithms.GRASP.GRASP import GRASP
from algorithms.genetic.nsgaii.nsgaii_algorithm import NSGAIIAlgorithm


fig, (ax1, ax2) = plt.subplots(1, 2)
#fig.suptitle("Dataset "+dataset)
#ax1.plot(x, y)
#ax2.plot(x, -y)


datasets=["1","2","s1","s2","s3"]
algorithms_data=[]
for dataset in datasets:
    labels=[]
    data_bar=[]
    sizes=[50,45,40,35,30,25]
    #dataset="2"
    seed=10
    generations=300
    solutions_per_iteration=500
    population_length=200
    algorithms = [
        GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
               init_type="uniform",local_search_type="None",
              path_relinking_mode="None"),
        GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
               init_type="stochastically",local_search_type="None",
              path_relinking_mode="None"),
        GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
               init_type="stochastically",local_search_type="None",
              path_relinking_mode="after_local"),
        #GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
        #       init_type="stochastically",local_search_type="best_first_neighbor_sorted_score",
        #      path_relinking_mode="None"),
        #GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
        #       init_type="stochastically",local_search_type="best_first_neighbor_sorted_score",
        #      path_relinking_mode="after_local"),
        NSGAIIAlgorithm(dataset_name=dataset,random_seed=seed,population_length=population_length,max_generations=generations,crossover_prob=0.8,
                               crossover="onepoint",mutation_prob=1.0,mutation="flip1bit",replacement="elitism"),
    ]
    for i in range(len(algorithms)):   
        algorithms_data.append([])
        print(algorithms[i].get_name())
        if "GRASP" in algorithms[i].file:
            pareto_file = "../output/output/pareto-grasp-"+algorithms[i].file
            metrics_file = "../output/metrics2307/pareto-grasp-"+algorithms[i].file
            df = findData(algorithms[i],"grasp")
            display(df)
        else:
            pareto_file = "../output/output/pareto-genetic-"+algorithms[i].file
            metrics_file = "../output/metrics2307/pareto-genetic-"+algorithms[i].file
            df = findData(algorithms[i],"genetic")
            display(df)
        data = np.loadtxt(pareto_file,delimiter=',', dtype=float)
        x,y=data.T
        #ax1.scatter(x,y,label=algorithms[i].get_name(),s=sizes[i])
        labels.append(algorithms[i].get_name())
        display(df["Time(s)"])
        data_bar.append(df["Time(s)"])
        algorithms_data[i].append(df["Time(s)"])
    #ax2.bar(["labels","a"],[4,2],width=0.2)
    print(labels)
    print(data_bar)
    #ax2.bar(labels,data_bar,width=0.2,label=dataset)
#ax2.bar_label(bar_ax, label_type='center')

for i in range(len(datasets)):
    ax2.bar(algorithms_data,width=0.2,label=dataset)
    
    
for i, v in enumerate(data_bar):
    plt.text(labels[i], v, str(v))

#ax1.set(xlabel='Coste', ylabel='Satisfacción')
#ax2.set(xlabel='Algoritmo', ylabel='Tiempo(s)')

#ax1.legend(loc="lower right")
#ax1.grid(True)
#plt.rcParams['figure.figsize'] = [36, 10]
#plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
#plt.show()




In [ ]:
from algorithms.GRASP.GRASP import GRASP
from algorithms.genetic.nsgaii.nsgaii_algorithm import NSGAIIAlgorithm

generations=300
solutions_per_iteration=500
population_length=200
seed=10
datasets=["1","2","s1","s2","s3"]
df_bar=pd.DataFrame()
series=[]
for dataset in datasets:
    algorithms = [
        GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
               init_type="uniform",local_search_type="None",
              path_relinking_mode="None"),
        GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
               init_type="stochastically",local_search_type="None",
              path_relinking_mode="None"),
        GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
               init_type="stochastically",local_search_type="None",
              path_relinking_mode="after_local"),
        GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
               init_type="stochastically",local_search_type="best_first_neighbor_sorted_score",
              path_relinking_mode="None"),
        GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
               init_type="stochastically",local_search_type="best_first_neighbor_sorted_score",
              path_relinking_mode="after_local"),
        NSGAIIAlgorithm(dataset_name=dataset,random_seed=seed,population_length=population_length,max_generations=generations,crossover_prob=0.8,
                               crossover="onepoint",mutation_prob=1.0,mutation="flip1bit",replacement="elitism"),
    ]
    for alg in algorithms:
        
        #print(alg.file)
        if "GRASP" in alg.file:
            df_aa=findData(alg,"grasp")
        else:
            df_aa=findData(alg,"genetic")
        df_aa["Algorithm"]=alg.get_name()
        df_aa["Dataset"]=dataset
        #display(df_aa)
        series.append(df_aa)
        df_bar.append(df_aa,ignore_index=True)
        #display(df_bar)

df_bar = pd.DataFrame(series,columns=series[0].index)
display(df_bar)
sns.barplot(x="Dataset", y="Time(s)", hue="Algorithm", data=df_bar, ci=None);

In [ ]:
sns.barplot(x="Dataset", y="HV", hue="Algorithm", data=df_bar, ci=None);

In [ ]:
sns.barplot(x="Dataset", y="Spread", hue="Algorithm", data=df_bar, ci=None);

In [ ]:
sns.barplot(x="Dataset", y="NumSolutions", hue="Algorithm", data=df_bar, ci=None);

In [ ]:
sns.barplot(x="Dataset", y="Spacing", hue="Algorithm", data=df_bar, ci=None);

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from algorithms.GRASP.GRASP import GRASP
from algorithms.genetic.nsgaii.nsgaii_algorithm import NSGAIIAlgorithm
sizes=[50,45,40,35,30,25]
dataset="s3"
seed=10
generations=300
solutions_per_iteration=500
population_length=200
algorithms = [
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="uniform",local_search_type="None",
          path_relinking_mode="None"),
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="stochastically",local_search_type="None",
          path_relinking_mode="None"),
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="stochastically",local_search_type="None",
          path_relinking_mode="after_local"),
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="stochastically",local_search_type="best_first_neighbor_sorted_score",
          path_relinking_mode="None"),
    GRASP(dataset=dataset,iterations=generations,solutions_per_iteration=solutions_per_iteration,seed=seed,
           init_type="stochastically",local_search_type="best_first_neighbor_sorted_score",
          path_relinking_mode="after_local"),
    NSGAIIAlgorithm(dataset_name=dataset,random_seed=seed,population_length=population_length,max_generations=generations,crossover_prob=0.8,
						   crossover="onepoint",mutation_prob=1.0,mutation="flip1bit",replacement="elitism"),
]

for i in range(len(algorithms)):   
    if "GRASP" in algorithms[i].file:
        file = "../output/output/pareto-grasp-"+algorithms[i].file
    else:
        file = "../output/output/pareto-genetic-"+algorithms[i].file
    data = np.loadtxt(file,delimiter=',', dtype=float)
    x,y=data.T
    plt.scatter(x,y,label=algorithms[i].get_name(),s=sizes[i])


#file = "output/backtracking.txt"
#data = np.loadtxt(file,delimiter=',', dtype=float)
#x,y=data.T
#plt.scatter(x,y,label="optimo",s=10)
plt.title(dataset)
plt.xlabel('Coste', fontsize=12)
plt.ylabel('Satisfacción', fontsize=12)
plt.legend(loc="lower right")
plt.title(dataset)
plt.grid(True)
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower
plt.show()

In [ ]:
display(findData("genetic"))

In [ ]:
from algorithms.backtracking_algorithm.backtracking_algorithm import BacktrackingAlgorithm
alg = BacktrackingAlgorithm(dataset="1")
alg.run()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
file = "algorithms/backtracking_algorithm/backtracking.txt"
data = np.loadtxt(file,delimiter=',', dtype=float)
x,y=data.T
plt.scatter(x,y)
plt.show()

In [ ]:
import numpy as np

a = np.array([[1,2,3], [2,2,3], [3,2,3]])
index = [0]

new_a = np.delete(a, index,axis=0)

print(new_a) #Prints `[1, 2, 5, 6, 8, 9]`

In [ ]:
df_a = df_alg[(df_alg["Algorithm"]=="NSGAIIAlgorithm")&(df_alg["Dataset"]=="2")&(df_alg["Replacement Scheme"]=="elitism")&(df_alg["Population Length"]==200)]
display(df_a)

In [1]:
from algorithms.genetic.geneticnds.geneticnds_algorithm import GeneticNDSAlgorithm
from algorithms.GRASP.GRASP import GRASP
import matplotlib.pyplot as plt
import pandas as pd
# crear seed random------------------------------------------------------------------
seed=54

# iniciar------------------------------------------------------------------
print("Running...")

algorithm=GRASP(dataset="2",seed=seed,iterations=30,solutions_per_iteration=50,init_type="stochastically",
                local_search_type="None")
result=algorithm.run()
func = [i.objectives for i in result["population"]]
function1 = [i[0].value for i in func]
function2 = [i[1].value for i in func]

algorithm=GRASP(dataset="2",seed=seed,iterations=30,solutions_per_iteration=50,init_type="stochastically",
                local_search_type="None",path_relinking_mode="after_local")
result=algorithm.run()
func = [i.objectives for i in result["population"]]
function3 = [i[0].value for i in func]
function4 = [i[1].value for i in func]

plt.xlabel('Coste', fontsize=12)
plt.ylabel('Satisfacción', fontsize=12)
plt.scatter(function2, function1, marker='o',label="GRASP-",c="#3cb371")
plt.scatter(function4, function3, marker='x',label="GRASP-PR",c="#000000")
plt.legend(loc="upper left")
fig1 = plt.gcf()
plt.show()
plt.draw()
#fig1.savefig('plots/dataset2_pareto.svg', dpi=100)

Running...

NDS created has 128 solution(s)


ValueError: max() arg is an empty sequence

In [ ]:
from algorithms.EDA.UMDA.umda_algorithm import UMDAAlgorithm
from algorithms.EDA.PBIL.pbil_algorithm import PBILAlgorithm
from algorithms.GRASP.GRASP import GRASP
from datasets.dataset1 import generate_dataset1_genes
from datasets.dataset2 import generate_dataset2_genes

from models.problem import Problem
import matplotlib.pyplot as plt
import pandas as pd
# crear seed random------------------------------------------------------------------
seed=54

# iniciar------------------------------------------------------------------
print("Running...")

algorithm=GRASP(dataset="s3",iterations=300,solutions_per_iteration=100,
        init_type="stochastically",local_search_type="None",path_relinking_mode="after_local")
result=algorithm.run()
func = [i.objectives for i in result["population"]]
function1 = [i[0].value for i in func]
function2 = [i[1].value for i in func]

#for i in func:
#    print(i[0].value,i[1].value)
    
algorithm=UMDAAlgorithm(dataset_name="s3",random_seed=seed,population_length=200,
                        max_generations=300,selected_individuals=120)
result=algorithm.run()
func = [i.objectives for i in result["population"]]
function3 = [i[0].value for i in func]
function4 = [i[1].value for i in func]

algorithm=PBILAlgorithm(dataset_name="s3",random_seed=seed,population_length=200,
            max_generations=300,learning_rate=0.1, mutation_prob=0.1, mutation_shift=0.1)
result=algorithm.run()
func = [i.objectives for i in result["population"]]
function5 = [i[0].value for i in func]
function6 = [i[1].value for i in func]

plt.xlabel('Coste', fontsize=12)
plt.ylabel('Satisfacción', fontsize=12)
plt.scatter(function2, function1, marker='o',label="GRASP",c="#3cb371")
plt.scatter(function4, function3, marker='x',label="UMDA",c="#000000")
plt.scatter(function6, function5, marker='+',label="PBIL",c="#cdcdcd")
plt.legend(loc="upper left")
fig1 = plt.gcf()
plt.show()
plt.draw()